In [ ]:
# Подключение Google Drive для доступа к данным
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineG

In [ ]:
# Загрузка необходимых библиотек
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch

# Трейн и тест

In [ ]:
# Определение путей к файлам
data_path = "./"
valid_file = data_path + "valid.csv"

import pandas as pd

# Загрузка данных
valid_path = "valid.csv"
train_path = "train.csv"

valid_data = pd.read_csv(valid_path)
train_data = pd.read_csv(train_path)

In [ ]:
valid_data

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,Это в Новом Орлеане?? Я действительно чувствую...,0,0,0,0,0,0,1
1,"Ты знаешь ответ, чувак, ты запрограммирован пе...",0,0,0,1,0,0,1
2,Мне никогда в жизни не было так грустно!,0,0,0,0,1,0,0
3,Экономика находится под жестким контролем и су...,0,0,0,1,0,0,1
4,Он мог бы легко взять настоящую камеру из зако...,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...
5421,"Это довольно опасно, когда государство решает,...",0,0,1,0,0,0,0
5422,"Сегодня утром я подала на развод. Надеюсь, он ...",0,0,0,1,0,0,0
5423,"В последний раз, когда это случилось, я просто...",1,0,0,0,0,0,0
5424,"Терпеть не могу этого высокомерного придурка, ...",1,0,0,0,0,0,0


In [ ]:
train_data

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,"Моя любимая еда — это все, что мне не приходил...",0,0,0,0,0,0,1
1,"Теперь, если он покончит с собой, все будут ду...",0,0,0,0,0,0,1
2,КАКОГО НАХРАНА БЭЙЛЕССКАЯ ИЗОИНЦИЯ?,1,0,0,0,0,0,0
3,Чтобы она почувствовала угрозу,0,0,1,0,0,0,0
4,Грязные южные дрочники,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
43405,"Добавил, приятель, у меня только что появился ...",0,0,0,1,0,0,0
43406,"Всегда думал, что это смешно, но является ли э...",0,0,0,0,0,1,0
43407,"О чем ты говоришь? Во всем плохом, что произош...",1,0,0,0,0,0,0
43408,"Больше похоже на крещение, с сексуальными резу...",0,0,0,1,0,0,0


In [ ]:
# Подсчет количества записей каждого класса в валидационном и тренировочном датасете
valid_class_counts = valid_data.iloc[:, 1:].sum()
train_class_counts = train_data.iloc[:, 1:].sum()

# Подсчет общего количества символов в классах sadness, fear и disgust
valid_char_counts = valid_data[valid_data['sadness'] == 1]['text'].str.len().sum() + \
                    valid_data[valid_data['fear'] == 1]['text'].str.len().sum() + \
                    valid_data[valid_data['disgust'] == 1]['text'].str.len().sum()

train_char_counts = train_data[train_data['sadness'] == 1]['text'].str.len().sum() + \
                    train_data[train_data['fear'] == 1]['text'].str.len().sum() + \
                    train_data[train_data['disgust'] == 1]['text'].str.len().sum()

valid_class_counts, train_class_counts, valid_char_counts, train_char_counts

(anger        717
 disgust       97
 fear         105
 joy         2219
 sadness      390
 surprise     624
 neutral     1766
 dtype: int64,
 anger        5579
 disgust       793
 fear          726
 joy         17410
 sadness      3263
 surprise     5367
 neutral     14219
 dtype: int64,
 47675,
 373343)

In [ ]:
# Загрузка данных из сохраненного файла
valid_data = train_data

# Проверка наличия единиц сразу в двух или более столбцах для каждого текста
multi_label_rows = valid_data.iloc[:, 1:].sum(axis=1) > 1

# Сколько таких строк
multi_label_count = multi_label_rows.sum()

# Примеры таких строк
multi_label_examples = valid_data[multi_label_rows]

multi_label_count, multi_label_examples.head()

(3855,
                                                  text  anger  disgust  fear  \
 15  Черт, кажется, я случайно купил боксерский пое...      1        0     0   
 20  Никогда не думал, что это происходит в один и ...      0        0     0   
 33       Хорошо, тогда какой, черт возьми, твой план?      1        0     0   
 42  Я читал в другом посте, что он умер вскоре пос...      0        0     0   
 44  Это безумие, как далеко зашел Photoshop. Подво...      0        0     0   
 
     joy  sadness  surprise  neutral  
 15    0        1         0        0  
 20    0        1         1        1  
 33    0        0         1        0  
 42    0        1         0        1  
 44    1        0         1        0  )

In [ ]:
# Фильтрация строк только с классом sadness
sadness_train = train_data[train_data['sadness'] == 1]['text']
sadness_valid = valid_data[valid_data['sadness'] == 1]['text']

# Объединение двух датасетов
combined_sadness = pd.concat([sadness_train, sadness_valid])

# Сохранение строк в один список для нарезки по 25,000 символов
combined_text = "\n".join(combined_sadness)

# Нарезка текста на части по 25,000 символов
chunk_size = 25000
text_chunks = [combined_text[i:i + chunk_size] for i in range(0, len(combined_text), chunk_size)]

# Сохранение частей в текстовые файлы
file_paths = []
for idx, chunk in enumerate(text_chunks):
    file_path = f"sadness_chunk_{idx + 1}.txt"
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(chunk)
    file_paths.append(file_path)

file_paths


['sadness_chunk_1.txt',
 'sadness_chunk_2.txt',
 'sadness_chunk_3.txt',
 'sadness_chunk_4.txt',
 'sadness_chunk_5.txt',
 'sadness_chunk_6.txt',
 'sadness_chunk_7.txt',
 'sadness_chunk_8.txt',
 'sadness_chunk_9.txt',
 'sadness_chunk_10.txt',
 'sadness_chunk_11.txt',
 'sadness_chunk_12.txt']

In [ ]:
# Фильтрация строк только с классом sadness
fear_train = train_data[train_data['fear'] == 1]['text']
fear_valid = valid_data[valid_data['fear'] == 1]['text']

# Объединение двух датасетов
combined_fear = pd.concat([fear_train, fear_valid])

# Сохранение строк в один список для нарезки по 25,000 символов
combined_text = "\n".join(combined_fear)

# Нарезка текста на части по 25,000 символов
chunk_size = 25000
text_chunks = [combined_text[i:i + chunk_size] for i in range(0, len(combined_text), chunk_size)]

# Сохранение частей в текстовые файлы
file_paths = []
for idx, chunk in enumerate(text_chunks):
    file_path = f"fear_chunk_{idx + 1}.txt"
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(chunk)
    file_paths.append(file_path)

file_paths


['fear_chunk_1.txt', 'fear_chunk_2.txt', 'fear_chunk_3.txt']

In [ ]:
# Фильтрация строк только с классом sadness
disgust_train = train_data[train_data['disgust'] == 1]['text']
disgust_valid = valid_data[valid_data['disgust'] == 1]['text']

# Объединение двух датасетов
combined_disgust = pd.concat([disgust_train, disgust_valid])

# Сохранение строк в один список для нарезки по 25,000 символов
combined_text = "\n".join(combined_disgust)

# Нарезка текста на части по 25,000 символов
chunk_size = 25000
text_chunks = [combined_text[i:i + chunk_size] for i in range(0, len(combined_text), chunk_size)]

# Сохранение частей в текстовые файлы
file_paths = []
for idx, chunk in enumerate(text_chunks):
    file_path = f"disgust_chunk_{idx + 1}.txt"
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(chunk)
    file_paths.append(file_path)

file_paths


['disgust_chunk_1.txt', 'disgust_chunk_2.txt', 'disgust_chunk_3.txt']

закидывалоь


# Djacon

In [ ]:
import pandas as pd

# Загружаем файлы
train = pd.read_csv("train_djacon.csv")
val = pd.read_csv("val_djacon.csv")
test = pd.read_csv("test_djacon.csv")

# Объединяем данные
combined_data = pd.concat([train, val, test], ignore_index=True)

# Создаем базовую структуру с нужными колонками
columns_to_keep = ["text", "anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
final_data = pd.DataFrame(columns=columns_to_keep)

# Копируем данные из исходного набора
final_data["text"] = combined_data["text"]
for col in ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]:
    if col in combined_data.columns:
        final_data[col] = combined_data[col]
    else:
        final_data[col] = 0  # Заполняем отсутствующие колонки нулями

# Проверяем результат
print(final_data.head())

# Сохраняем итоговый датасет
output_path = "processed_djacon.csv"
final_data.to_csv(output_path, index=False)
print(f"Файл сохранён как {output_path}.")


                                                text  anger  disgust  fear  \
0                                           Ты дебил      1        1     0   
1  Я чувствую себя так глупо. Я никогда не видел ...      1        1     0   
2  Это правда, их послужной список не очень хорош...      1        0     0   
3  Это факт. Так что ваши обвинения просто основа...      1        0     0   
4  Моему сыну 10 месяцев. Я надеюсь, что у нас бу...      0        0     0   

   joy  sadness  surprise  neutral  
0    0        0         0        0  
1    0        0         0        0  
2    0        0         0        0  
3    0        0         0        0  
4    1        0         0        0  
Файл сохранён как processed_djacon.csv.


In [ ]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24891 entries, 0 to 24890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      24891 non-null  object
 1   anger     24891 non-null  int64 
 2   disgust   24891 non-null  int64 
 3   fear      24891 non-null  int64 
 4   joy       24891 non-null  int64 
 5   sadness   24891 non-null  int64 
 6   surprise  24891 non-null  int64 
 7   neutral   24891 non-null  int64 
dtypes: int64(7), object(1)
memory usage: 1.5+ MB


# датасет DUSHA

In [ ]:
# Путь к новому сбалансированному датасету
dusha_metadata_path = '/content/drive/MyDrive/EchoStressAI/Development/dusha_dataset/data/crowd/crowd_test/raw_crowd_test.tsv'

# Загрузка сбалансированного датасета
dusha_test_metadata = pd.read_csv(dusha_metadata_path, sep='\t')

# Проверка, что данные загрузились корректно
print("Распределение аудиофайлов в тестовом датасете Dusha:")
print(dusha_test_metadata['annotator_emo'].value_counts())

Распределение аудиофайлов в тестовом датасете Dusha:
annotator_emo
neutral     48917
sad         17603
angry        5987
positive     5775
other         806
Name: count, dtype: int64


In [ ]:
dusha_test_metadata

,hash_id,audio_path,duration,annotator_emo,golden_emo,annotator_id,speaker_text,speaker_emo,source_id
0,9e9961c53ca6eeb440b217e539fbf46c,wavs/9e9961c53ca6eeb440b217e539fbf46c.wav,5.82000,neutral,NaN,858305a5450b7bd1288ba0053b1cd1c1,я слушаю,neutral,4282ddc30d71ef420e202e0c60391e9f
1,0166f65a30354db8282682b1a280e64c,wavs/0166f65a30354db8282682b1a280e64c.wav,3.70000,sad,NaN,858305a5450b7bd1288ba0053b1cd1c1,каким стал сбер,neutral,d70dc98ed56e9362eaefefb7b2827c8f
2,d49a6b560155831725a7bdc7d0a96099,wavs/d49a6b560155831725a7bdc7d0a96099.wav,4.38000,neutral,NaN,858305a5450b7bd1288ba0053b1cd1c1,где родился шерлок холмс,neutral,0ee35d2abecf4272ecc8e1539b0839d8
3,c6852b0925797612d7b6724da8cbe7b4,wavs/c6852b0925797612d7b6724da8cbe7b4.wav,8.58000,neutral,NaN,858305a5450b7bd1288ba0053b1cd1c1,открой в браузере ennio morricone,neutral,0855e363c1787df1592f58f7a27ebe13
4,0166f65a30354db8282682b1a280e64c,wavs/0166f65a30354db8282682b1a280e64c.wav,3.70000,sad,NaN,a5562e26cd8f1949488a2d1e1e549d97,каким стал сбер,neutral,d70dc98ed56e9362eaefefb7b2827c8f
...,...,...,...,...,...,...,...,...,...
79083,6c0dddce6a86f7fe9c017c03a299773e,wavs/6c0dddce6a86f7fe9c017c03a299773e.wav,4.92000,sad,3.0,f37940680d604692ea541b439fa39762,NaN,NaN,NaN
79084,d893d560e540c0c34de1ec9a779bde67,wavs/d893d560e540c0c34de1ec9a779bde67.wav,3.86000,neutral,2.0,3f2111910c469da2869d8f4178c4a945,NaN,NaN,NaN
79085,99baaf6ed01355a799311d49121e0a0f,wavs/99baaf6ed01355a799311d49121e0a0f.wav,7.24425,sad,3.0,b9413991d11e08bb636c1ce2aeedb1c2,NaN,NaN,NaN
79086,aca882b01aa79b7a75bd2e8acb2373f9,wavs/aca882b01aa79b7a75bd2e8acb2373f9.wav,4.92000,sad,3.0,0f138ca8a0ce058f48d47cda85753439,NaN,NaN,NaN


In [ ]:
# Фильтрация строк только с классом sadness
sadness_dusha_test = dusha_test_metadata[dusha_test_metadata['annotator_emo'] == 'sad']['speaker_text']

In [ ]:
sadness_dusha_test

,speaker_text
1,каким стал сбер
4,каким стал сбер
8,каким стал сбер
18,позови мне кого нибудь другого лезть
36,у меня зуб выпадал
...,...
79082,NaN
79083,NaN
79085,NaN
79086,NaN


In [ ]:
# Удаление пустых строк
sadness_dusha_test = sadness_dusha_test.dropna()

# Удаление повторяющихся строк
sadness_dusha_test = sadness_dusha_test.drop_duplicates()

# Подсчет оставшихся строк
remaining_rows = sadness_dusha_test.shape[0]

print (remaining_rows)

5230


In [ ]:
sadness_dusha_test

,speaker_text
1,каким стал сбер
18,позови мне кого нибудь другого лезть
36,у меня зуб выпадал
38,прости и гость не стал для
53,мне не нравится мне нравится
...,...
77807,доктор плюшева доктор плюшева
77808,включи песню бабник
77810,виски кола королева танцпола
77822,расскажи пош


In [ ]:
data = sadness_dusha_test
data

,speaker_text
1,каким стал сбер
18,позови мне кого нибудь другого лезть
36,у меня зуб выпадал
38,прости и гость не стал для
53,мне не нравится мне нравится
...,...
77807,доктор плюшева доктор плюшева
77808,включи песню бабник
77810,виски кола королева танцпола
77822,расскажи пош


In [ ]:
# Преобразование Series в DataFrame
sadness_dusha_test_df = sadness_dusha_test.to_frame()

# Проверка доступных столбцов
columns = sadness_dusha_test_df.columns
columns


Index(['speaker_text'], dtype='object')

In [ ]:
# Преобразование Series в DataFrame с явным именем колонки
sadness_dusha_test_df = sadness_dusha_test.to_frame(name='speaker_text')

In [ ]:
sadness_dusha_test_df.columns

Index(['speaker_text'], dtype='object')

In [ ]:
# Подготовка строк для записи в текстовые файлы
combined_text = "\n".join(sadness_dusha_test_df['speaker_text'])

# Нарезка текста на части по 25,000 символов
chunk_size = 25000
text_chunks = [combined_text[i:i + chunk_size] for i in range(0, len(combined_text), chunk_size)]

# Сохранение частей в текстовые файлы
file_paths = []
for idx, chunk in enumerate(text_chunks):
    file_path = f"sadness_dusha_chunk_{idx + 1}.txt"
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(chunk)
    file_paths.append(file_path)

# Подсчет оставшихся строк
remaining_rows = sadness_dusha_test_df.shape[0]

remaining_rows, file_paths


(5230,
 ['sadness_dusha_chunk_1.txt',
  'sadness_dusha_chunk_2.txt',
  'sadness_dusha_chunk_3.txt',
  'sadness_dusha_chunk_4.txt',
  'sadness_dusha_chunk_5.txt'])

In [ ]:
# from transformers import pipeline
# data = sadness_dusha_test_df
# text_column = 'speaker_text'

# # Инициализируем модель для анализа тональности
# emotion_analyzer = pipeline("sentiment-analysis", model="mrm8488/t5-base-finetuned-emotion", return_all_scores=True)

# # Функция для фильтрации строк с грустным оттенком
# def detect_sadness(text):
#     try:
#         # Получаем предсказания эмоций
#         predictions = emotion_analyzer(text)
#         # Ищем вероятность грусти
#         for pred in predictions[0]:
#             if pred['label'] in ['sadness', 'depression', 'despair'] and pred['score'] >= 0.55:
#                 return True
#         return False
#     except Exception as e:
#         print(f"Ошибка при обработке текста: {text}, {e}")
#         return False

# # Применяем функцию фильтрации к датасету
# filtered_data = data[data[text_column].apply(detect_sadness)]

# # Сохраняем отфильтрованные данные в CSV
# filtered_data.to_csv('filtered_sad_phrases.csv', index=False)

# # Скачиваем результат
# files.download('filtered_sad_phrases.csv')

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at mrm8488/t5-base-finetuned-emotion and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
data

,speaker_text
1,каким стал сбер
18,позови мне кого нибудь другого лезть
36,у меня зуб выпадал
38,прости и гость не стал для
53,мне не нравится мне нравится
...,...
77807,доктор плюшева доктор плюшева
77808,включи песню бабник
77810,виски кола королева танцпола
77822,расскажи пош


In [ ]:
# Путь к датасету
dusha_metadata_path = '/content/drive/MyDrive/EchoStressAI/Development/dusha_dataset/data/crowd/crowd_train/raw_crowd_train.tsv'

# Загрузка сбалансированного датасета
dusha_train_metadata = pd.read_csv(dusha_metadata_path, sep='\t')

# Проверка, что данные загрузились корректно
print("Распределение аудиофайлов в тестовом датасете Dusha:")
print(dusha_train_metadata['annotator_emo'].value_counts())

Распределение аудиофайлов в тестовом датасете Dusha:
annotator_emo
neutral     559501
sad         164671
positive     87258
angry        85527
other         9996
Name: count, dtype: int64


In [ ]:
# Фильтрация строк только с классом sadness
sadness_dusha_train = dusha_train_metadata[dusha_train_metadata['annotator_emo'] == 'sad']['speaker_text']

# Удаление пустых строк
sadness_dusha_train = sadness_dusha_train.dropna()

# Удаление повторяющихся строк
sadness_dusha_train = sadness_dusha_train.drop_duplicates()

# Подсчет оставшихся строк
remaining_rows = sadness_dusha_train.shape[0]

print (remaining_rows)

46364


In [ ]:
# Преобразование Series в DataFrame с явным именем колонки
sadness_dusha_train = sadness_dusha_train.to_frame(name='speaker_text')

In [ ]:
# Подготовка строк для записи в текстовые файлы
combined_text = "\n".join(sadness_dusha_train['speaker_text'])

# Нарезка текста на части по 25,000 символов
chunk_size = 25000
text_chunks = [combined_text[i:i + chunk_size] for i in range(0, len(combined_text), chunk_size)]

# Сохранение частей в текстовые файлы
file_paths = []
for idx, chunk in enumerate(text_chunks):
    file_path = f"sadness_dusha_train_chunk_{idx + 1}.txt"
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(chunk)
    file_paths.append(file_path)

# Подсчет оставшихся строк
remaining_rows = sadness_dusha_test_df.shape[0]

remaining_rows, file_paths

(5230,
 ['sadness_dusha_train_chunk_1.txt',
  'sadness_dusha_train_chunk_2.txt',
  'sadness_dusha_train_chunk_3.txt',
  'sadness_dusha_train_chunk_4.txt',
  'sadness_dusha_train_chunk_5.txt',
  'sadness_dusha_train_chunk_6.txt',
  'sadness_dusha_train_chunk_7.txt',
  'sadness_dusha_train_chunk_8.txt',
  'sadness_dusha_train_chunk_9.txt',
  'sadness_dusha_train_chunk_10.txt',
  'sadness_dusha_train_chunk_11.txt',
  'sadness_dusha_train_chunk_12.txt',
  'sadness_dusha_train_chunk_13.txt',
  'sadness_dusha_train_chunk_14.txt',
  'sadness_dusha_train_chunk_15.txt',
  'sadness_dusha_train_chunk_16.txt',
  'sadness_dusha_train_chunk_17.txt',
  'sadness_dusha_train_chunk_18.txt',
  'sadness_dusha_train_chunk_19.txt',
  'sadness_dusha_train_chunk_20.txt',
  'sadness_dusha_train_chunk_21.txt',
  'sadness_dusha_train_chunk_22.txt',
  'sadness_dusha_train_chunk_23.txt',
  'sadness_dusha_train_chunk_24.txt',
  'sadness_dusha_train_chunk_25.txt',
  'sadness_dusha_train_chunk_26.txt',
  'sadness_dus

In [ ]:
import zipfile
import os

# Путь для сохранения архива
archive_name = "sadness_dusha_train_chunks.zip"

# Список файлов для архивации
file_paths = [
    'sadness_dusha_train_chunk_1.txt',
    'sadness_dusha_train_chunk_2.txt',
    'sadness_dusha_train_chunk_3.txt',
    'sadness_dusha_train_chunk_4.txt',
    'sadness_dusha_train_chunk_5.txt',
    'sadness_dusha_train_chunk_6.txt',
    'sadness_dusha_train_chunk_7.txt',
    'sadness_dusha_train_chunk_8.txt',
    'sadness_dusha_train_chunk_9.txt',
    'sadness_dusha_train_chunk_10.txt',
    'sadness_dusha_train_chunk_11.txt',
    'sadness_dusha_train_chunk_12.txt',
    'sadness_dusha_train_chunk_13.txt',
    'sadness_dusha_train_chunk_14.txt',
    'sadness_dusha_train_chunk_15.txt',
    'sadness_dusha_train_chunk_16.txt',
    'sadness_dusha_train_chunk_17.txt',
    'sadness_dusha_train_chunk_18.txt',
    'sadness_dusha_train_chunk_19.txt',
    'sadness_dusha_train_chunk_20.txt',
    'sadness_dusha_train_chunk_21.txt',
    'sadness_dusha_train_chunk_22.txt',
    'sadness_dusha_train_chunk_23.txt',
    'sadness_dusha_train_chunk_24.txt',
    'sadness_dusha_train_chunk_25.txt',
    'sadness_dusha_train_chunk_26.txt',
    'sadness_dusha_train_chunk_27.txt',
    'sadness_dusha_train_chunk_28.txt',
    'sadness_dusha_train_chunk_29.txt',
    'sadness_dusha_train_chunk_30.txt',
    'sadness_dusha_train_chunk_31.txt',
    'sadness_dusha_train_chunk_32.txt',
    'sadness_dusha_train_chunk_33.txt',
    'sadness_dusha_train_chunk_34.txt',
    'sadness_dusha_train_chunk_35.txt',
    'sadness_dusha_train_chunk_36.txt',
    'sadness_dusha_train_chunk_37.txt',
    'sadness_dusha_train_chunk_38.txt',
    'sadness_dusha_train_chunk_39.txt',
    'sadness_dusha_train_chunk_40.txt',
    'sadness_dusha_train_chunk_41.txt',
    'sadness_dusha_train_chunk_42.txt',
    'sadness_dusha_train_chunk_43.txt',
    'sadness_dusha_train_chunk_44.txt',
]

# Создание архива
with zipfile.ZipFile(archive_name, 'w') as zipf:
    for file_path in file_paths:
        zipf.write(file_path, os.path.basename(file_path))

# Вывод пути к архиву
archive_name


'sadness_dusha_train_chunks.zip'

# Другие датасеты

In [ ]:
import pandas as pd
import glob

# Загрузка всех файлов с именами, начинающимися на 'IRAhandle_tweets_' и заканчивающимися на '.csv'
file_paths = sorted(glob.glob("IRAhandle_tweets_*.csv"))

# Проверка, какие файлы обнаружены
file_paths


['IRAhandle_tweets_1.csv',
 'IRAhandle_tweets_10.csv',
 'IRAhandle_tweets_11.csv',
 'IRAhandle_tweets_12.csv',
 'IRAhandle_tweets_13.csv',
 'IRAhandle_tweets_2.csv',
 'IRAhandle_tweets_3.csv',
 'IRAhandle_tweets_4.csv',
 'IRAhandle_tweets_5.csv',
 'IRAhandle_tweets_6.csv',
 'IRAhandle_tweets_7.csv',
 'IRAhandle_tweets_8.csv',
 'IRAhandle_tweets_9.csv']

In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Загружаем найденные файлы в список DataFrame
file_paths = [
 'IRAhandle_tweets_1.csv',
 'IRAhandle_tweets_10.csv',
 'IRAhandle_tweets_11.csv',
 'IRAhandle_tweets_12.csv',
 'IRAhandle_tweets_13.csv',
 'IRAhandle_tweets_2.csv',
 'IRAhandle_tweets_3.csv',
 'IRAhandle_tweets_4.csv',
 'IRAhandle_tweets_5.csv',
 'IRAhandle_tweets_6.csv',
 'IRAhandle_tweets_7.csv',
 'IRAhandle_tweets_8.csv',
 'IRAhandle_tweets_9.csv'
]

# Чтение и объединение файлов в один DataFrame
dataframes = [pd.read_csv(file) for file in file_paths]
combined_df = pd.concat(dataframes, ignore_index=True)


<ipython-input-9-3cd68b68206b>:19: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(file) for file in file_paths]
<ipython-input-9-3cd68b68206b>:19: DtypeWarning: Columns (10,20) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(file) for file in file_paths]
<ipython-input-9-3cd68b68206b>:19: DtypeWarning: Columns (0,15,20) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = [pd.read_csv(file) for file in file_paths]


ModuleNotFoundError: No module named 'ace_tools'

In [ ]:
combined_df

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,...,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,...,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,...,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,...,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,...,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2946202,2260338140,POLITICS_T0DAY,Порошенко призвал к полной перезагрузке правит...,United States,Russian,2/16/2016 22:20,2/16/2016 22:20,92,887,12934,...,Russian,0,NonEnglish,0,2260338140,699720021396869121,http://twitter.com/politics_t0day/statuses/699...,https://twitter.com/politics_t0day/status/6997...,http://www.interfax.ru/world/494930,NaN
2946203,2260338140,POLITICS_T0DAY,Турецкая атомная бомба появится в 2023 году. ...,United States,Russian,2/16/2016 22:25,2/16/2016 22:25,92,887,12935,...,Russian,0,NonEnglish,0,2260338140,699721241553432576,http://twitter.com/politics_t0day/statuses/699...,https://twitter.com/politics_t0day/status/6997...,http://svpressa.ru/war21/article/142481/,NaN
2946204,2260338140,POLITICS_T0DAY,История одной татуировки. Тату на предплечье:...,United States,Russian,2/16/2016 22:31,2/16/2016 22:31,92,887,12936,...,Russian,0,NonEnglish,0,2260338140,699722743533674496,http://twitter.com/politics_t0day/statuses/699...,https://twitter.com/politics_t0day/status/6997...,http://manzal.livejournal.com/359548.html,NaN
2946205,2260338140,POLITICS_T0DAY,"Индонезия, Китай, золото и isis #Эксклюзив@pol...",United States,Russian,2/16/2016 22:35,2/16/2016 22:35,92,887,12937,...,Russian,0,NonEnglish,0,2260338140,699723798849916928,http://twitter.com/politics_t0day/statuses/699...,https://twitter.com/politics_t0day/status/6997...,NaN,NaN


In [ ]:
combined_df.columns

Index(['external_author_id', 'author', 'content', 'region', 'language',
       'publish_date', 'harvested_date', 'following', 'followers', 'updates',
       'post_type', 'account_type', 'retweet', 'account_category',
       'new_june_2018', 'alt_external_id', 'tweet_id', 'article_url',
       'tco1_step1', 'tco2_step1', 'tco3_step1'],
      dtype='object')

In [ ]:
pip install datasets


In [ ]:
# Загрузка датасета DailyDialog с учетом remote code
dataset = load_dataset("daily_dialog", trust_remote_code=True)



In [ ]:
sample_example = dataset["train"][0]
print(sample_example)


{'dialog': ['Say , Jim , how about going for a few beers after dinner ? ', ' You know that is tempting but is really not good for our fitness . ', ' What do you mean ? It will help us to relax . ', " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ", " I guess you are right.But what shall we do ? I don't feel like sitting at home . ", ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ', " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ", ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ', " Good.Let ' s go now . ", ' All right . '], 'act': [3, 4, 2, 2, 2, 3, 4, 1, 3, 4], 'emotion': [0, 0, 0, 0, 0, 0, 4, 4, 4, 4]}


In [ ]:
# Фильтрация реплик по нужным эмоциям
filtered_samples = []

# Проходим по каждому диалогу в наборе данных
for dialog_entry in dataset["train"]:
    dialog = dialog_entry["dialog"]
    emotions = dialog_entry["emotion"]
    for text, emotion in zip(dialog, emotions):
        if emotion in desired_classes:  # Фильтруем по нужным эмоциям
            filtered_samples.append({"dialog": text, "emotion": emotion})

# Преобразуем результат в DataFrame для удобного просмотра
filtered_df = pd.DataFrame(filtered_samples)

# Вывод первых строк
filtered_df.head()


,dialog,emotion
0,"Oh , no !",2
1,How dreadful !,2
2,"Oh , dear . That ’ s terrible . Not at all wh...",2
3,"It ’ s awful , isn ’ t it ?",2
4,I can ’ t bear to look at it . I ’ m rather d...,2


In [ ]:
!pip install transformers


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Маппинг классов эмоций на текстовые метки
emotion_map = {1: "fear", 2: "disgust", 5: "sadness"}
filtered_df["emotion"] = filtered_df["emotion"].map(emotion_map)

# Инициализация модели для перевода
model_name = "Helsinki-NLP/opus-mt-en-ru"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Перевод текста на русский язык
def translate_to_russian(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return translated_text[0]

# Применение перевода к репликам
filtered_df["dialog"] = filtered_df["dialog"].apply(translate_to_russian)

# Вывод первых строк после обработки
filtered_df.head()


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

,dialog,emotion
0,"О, нет!",disgust
1,Как ужасно!,disgust
2,"О, дорогая, это ужасно, совсем не то, что я ож...",disgust
3,"Это ужасно, не так ли?",disgust
4,"Я могу стерпеть, чтобы посмотреть на это. Я оч...",disgust


# Применение модели

In [ ]:
# Инициализация предобученной модели для анализа текста
# Используем модель "badmatr11x/roberta-base-emotions-detection-from-text"
pretrained_model_name = "badmatr11x/roberta-base-emotions-detection-from-text"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name)

# Создаем pipeline для классификации текста
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, framework="pt", device=0 if torch.cuda.is_available() else -1)

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

In [ ]:
# Создаем итоговый датасет с нужными эмоциями
filtered_rows = []
for idx, row in dusha_test_metadata.iterrows():
    text = row['speaker_text']
    original_emotion = row['annotator_emo'].lower()
    prediction = classifier(text, top_k=1)[0]  # Получаем предсказание от модели
    predicted_emotion = prediction['label'].lower()

    # Условия для добавления строки в итоговый датасет
    if predicted_emotion == "sadness" and original_emotion == "sad":
        filtered_rows.append(row)
    elif predicted_emotion in ["fear", "disgust"]:
        filtered_rows.append(row)

# Создаем датафрейм с отфильтрованными строками
filtered_dataset = pd.DataFrame(filtered_rows)

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
# Создаем датафрейм с отфильтрованными строками
filtered_dataset = pd.DataFrame(filtered_rows)

# Сохраняем новый датасет в формате CSV
filtered_dataset_path = '/content/drive/MyDrive/EchoStressAI/Development/dusha_dataset/data/processed/filtered_emotions_dataset.csv'
filtered_dataset.to_csv(filtered_dataset_path, index=False)

print(f"Сохранено {len(filtered_rows)} строк в новый датасет по пути: {filtered_dataset_path}")

In [ ]:
# Выводим распределение эмоций в итоговом датасете
print("Распределение эмоций в итоговом датасете:")
print(filtered_dataset['annotator_emo'].value_counts())